In [1]:
mkdir ~/. kaggle

mkdir: cannot create directory ‘/root/.’: File exists
mkdir: cannot create directory ‘kaggle’: File exists


In [2]:
cp kaggle.json ~/. kaggle/

cp: cannot stat 'kaggle.json': No such file or directory
cp: -r not specified; omitting directory '/root/.'


In [3]:
! chmod ~/.kaggle/kaggle.json

chmod: missing operand after ‘/root/.kaggle/kaggle.json’
Try 'chmod --help' for more information.


In [4]:
! kaggle datasets download -d manjilkarki/deepfake-and-real-images

Dataset URL: https://www.kaggle.com/datasets/manjilkarki/deepfake-and-real-images
License(s): unknown
deepfake-and-real-images.zip: Skipping, found more recently modified local copy (use --force to force download)


In [5]:
! unzip /content/deepfake-and-real-images.zip

Archive:  /content/deepfake-and-real-images.zip
replace Dataset/Test/Fake/fake_0.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.metrics import F1Score

In [3]:
imgen = ImageDataGenerator(rescale=1/255)

In [4]:
train = imgen.flow_from_directory('/content/Dataset/Train',
                                  target_size=(224,224),
                                  class_mode='binary',
                                  seed=43)
test = imgen.flow_from_directory('/content/Dataset/Test',
                                  target_size=(224,224),
                                  class_mode='binary',
                                  seed=43)
valid = imgen.flow_from_directory('/content/Dataset/Validation',
                                  target_size=(224,224),
                                  class_mode='binary',
                                  seed=43)

Found 140002 images belonging to 2 classes.
Found 10905 images belonging to 2 classes.
Found 39428 images belonging to 2 classes.


In [5]:
model = Sequential()


model.add(Conv2D(filters=64,kernel_size=(3,3),activation='relu', input_shape=(224,224,3)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(filters=128,kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(filters=256,kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=[tf.keras.metrics.BinaryAccuracy()])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
model.fit(train,epochs=10,batch_size=32,validation_data=valid, steps_per_epoch=150, callbacks=EarlyStopping(patience=5, restore_best_weights=True))

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


150/150 ━━━━━━━━━━━━━━━━━━━━ 107s 641ms/step - binary_accuracy: 0.5334 - loss: 0.6904 - val_binary_accuracy: 0.5778 - val_loss: 0.6714
Epoch 2/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 98s 657ms/step - binary_accuracy: 0.5876 - loss: 0.6669 - val_binary_accuracy: 0.5885 - val_loss: 0.6555
Epoch 3/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 79s 527ms/step - binary_accuracy: 0.6525 - loss: 0.6282 - val_binary_accuracy: 0.6555 - val_loss: 0.6223
Epoch 4/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 73s 488ms/step - binary_accuracy: 0.6693 - loss: 0.6013 - val_binary_accuracy: 0.6792 - val_loss: 0.5976
Epoch 5/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 98s 655ms/step - binary_accuracy: 0.6994 - loss: 0.5825 - val_binary_accuracy: 0.6703 - val_loss: 0.6000
Epoch 6/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 98s 654ms/step - binary_accuracy: 0.7064 - loss: 0.5623 - val_binary_accuracy: 0.6124 - val_loss: 0.7089
Epoch 7/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 76s 509ms/step - binary_accuracy: 0.7228 - loss: 0.5434 - val_binary_accuracy: 0.7094 - val_loss: 0.

In [7]:
model.evaluate(test)

341/341 ━━━━━━━━━━━━━━━━━━━━ 25s 74ms/step - binary_accuracy: 0.7120 - loss: 0.5550


[0.5552656054496765, 0.7125172019004822]